In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bike_rentals = pd.read_csv("bike_rental_hour.csv")

On first glance....

In [3]:
bike_rentals.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
bike_rentals["cnt"].plot.hist(grid=True, bins = 5, rwidth = 0.9, color = '#607c8e')
plt.xlabel("Total Number of Bike Rentals per We")
plt.grid(axis = 'y', alpha = 0.75)

In [5]:
bike_rentals.corr()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,1.000000,0.404046,0.866014,0.489164,-0.004775,0.014723,0.001357,-0.003416,-0.014198,0.136178,0.137615,0.009577,-0.074505,0.158295,0.282046,0.278379
season,0.404046,1.000000,-0.010742,0.830386,-0.006117,-0.009585,-0.002335,0.013743,-0.014524,0.312025,0.319380,0.150625,-0.149773,0.120206,0.174226,0.178056
yr,0.866014,-0.010742,1.000000,-0.010473,-0.003867,0.006692,-0.004485,-0.002196,-0.019157,0.040913,0.039222,-0.083546,-0.008740,0.142779,0.253684,0.250495
mnth,0.489164,0.830386,-0.010473,1.000000,-0.005772,0.018430,0.010400,-0.003477,0.005400,0.201691,0.208096,0.164411,-0.135386,0.068457,0.122273,0.120638
hr,-0.004775,-0.006117,-0.003867,-0.005772,1.000000,0.000479,-0.003498,0.002285,-0.020203,0.137603,0.133750,-0.276498,0.137252,0.301202,0.374141,0.394071
holiday,0.014723,-0.009585,0.006692,0.018430,0.000479,1.000000,-0.102088,-0.252471,-0.017036,-0.027340,-0.030973,-0.010588,0.003988,0.031564,-0.047345,-0.030927
weekday,0.001357,-0.002335,-0.004485,0.010400,-0.003498,-0.102088,1.000000,0.035955,0.003311,-0.001795,-0.008821,-0.037158,0.011502,0.032721,0.021578,0.026900
workingday,-0.003416,0.013743,-0.002196,-0.003477,0.002285,-0.252471,0.035955,1.000000,0.044672,0.055390,0.054667,0.015688,-0.011830,-0.300942,0.134326,0.030284
weathersit,-0.014198,-0.014524,-0.019157,0.005400,-0.020203,-0.017036,0.003311,0.044672,1.000000,-0.102640,-0.105563,0.418130,0.026226,-0.152628,-0.120966,-0.142426
temp,0.136178,0.312025,0.040913,0.201691,0.137603,-0.027340,-0.001795,0.055390,-0.102640,1.000000,0.987672,-0.069881,-0.023125,0.459616,0.335361,0.404772


In [6]:
def assign_label(hour):
    if 6 < hour <= 12: 
        return 1
    elif 12 < hour <= 18:
        return 2
    elif 18 < hour < 24:
        return 3
    elif 0 <= hour <= 6:
        return 4

In [7]:
bike_rentals["time_label"] = bike_rentals["hr"].apply(assign_label)

In [8]:
bike_rentals.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,time_label
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,4
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,4
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,4
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,4
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,4


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LinearRegression

I selected the mean squared error as the test metric this time, because we are going to be predicting a continuous variable using regression, and mean_squared_error is a reliable metric for regression type problems. 

In [10]:
np.random.seed(101214)

train, test = train_test_split(bike_rentals,train_size = 0.8)

In [11]:
columns = ["mnth", "yr", "weekday", "temp", "atemp", "hum", "season"]

lr = LinearRegression()
lr.fit(train[columns], train["cnt"])

predictions = lr.predict(test[columns])

rmse = np.sqrt(mean_squared_error(predictions, test["cnt"]))
print(rmse)

149.42795408432147


The error is quite high for an rmse value. It seems like the linear regression is not predicting the values very well....maybe a non linear model is needed? Let's compared it to an error statistic provided by a DecisionTreeRegressor model.

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
dc_tree = DecisionTreeRegressor()
dc_tree.fit(train[columns], train["cnt"])

predictions = dc_tree.predict(test[columns])
rmse = np.sqrt(mean_squared_error(test['cnt'], predictions))
print(rmse)

171.56690624042437


In [ ]:
dc_tree = DecisionTreeRegressor()
grid = GridSearchCV(estimator = dc_tree, param_grid= {
    "min_samples_split": [2, 3, 4, 5],
    "splitter": ["best", "random"],
    "max_depth": [None, 1, 2, 3, 4],
    "min_samples_leaf": [1, 2, 3, 4],
    "max_features": ["sqrt", "auto", 2, 3, 4],
    "max_leaf_nodes":[2, 3, 4]
})
grid.fit(train[columns], train["cnt"])

In [ ]:
dc_tree = grid.best_estimator_
dc_tree.fit(train[columns], train["cnt"])

predictions = dc_tree.predict(test[columns])
rmse = np.sqrt(mean_squared_error(test['cnt'], predictions))
print(rmse)

In [ ]:
rf = RandomForestRegressor()
grid = GridSearchCV(estimator = rf, param_grid= {
    "n_estimators": [100,500,1000],
    "min_samples_split": [2, 3, 4, 5],
    "criterion": ["mse", "mae"],
    "max_depth" : [None, 1, 2, 3, 4],
    "min_samples_leaf" : [1, 2, 3, 4],
    "min_weight_fraction_leaf": [0.05, 0.10, 0.],
    "max_features" : ["sqrt", "auto", 2, 3, 4],
    "max_leaf_nodes" : [2, 3, 4]
})
grid.fit(train[columns], train["cnt"])

In [ ]:
rf = grid.best_estimator_
rf.fit(train[columns], train["cnt"])

predictions = rf.predict(test[columns])
rmse = np.sqrt(mean_squared_error(test['cnt'], predictions))
print(rmse)